# Intégration d'Apprentissage Contrastif dans DialogueGCN

## 1. Ajout d'une Loss Contrastive (Nouvelle classe)

```python
class ContrastiveLoss(nn.Module):   
    def __init__(self, margin=1.0, temperature=0.5):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.temperature = temperature
        self.cos_sim = nn.CosineSimilarity(dim=2)
        
    def forward(self, features, labels):
        """
        features: tensor of shape (batch_size, hidden_dim)
        labels: tensor of shape (batch_size)
        """
        # Normalize features
        features = F.normalize(features, p=2, dim=1)
        
        # Compute similarity matrix
        sim_matrix = torch.matmul(features, features.T) / self.temperature
        
        # Create mask for positive pairs (same class)
        label_matrix = labels.unsqueeze(1) == labels.unsqueeze(0)
        pos_mask = label_matrix.fill_diagonal_(False)  # Exclude self-pairs
        
        # Create mask for negative pairs (different classes)
        neg_mask = ~label_matrix
        
        # Compute positive and negative similarities
        pos_sim = sim_matrix[pos_mask]
        neg_sim = sim_matrix[neg_mask]
        
        # Contrastive loss
        loss = -torch.log(
            torch.exp(pos_sim).sum() / 
            (torch.exp(pos_sim).sum() + torch.exp(neg_sim).sum() + 1e-8)
        )
        
        return loss
```

## 2. Modification du GraphNetwork (lignes 723-875)

```python
class GraphNetwork(torch.nn.Module):
    def __init__(self, num_features, num_classes, num_relations, max_seq_len, 
                 hidden_size=64, dropout=0.5, no_cuda=False, use_contrastive=False):
        super(GraphNetwork, self).__init__()
        self.use_contrastive = use_contrastive
        
        self.conv1 = RGCNConv(num_features, hidden_size, num_relations, num_bases=30)
        self.conv2 = GraphConv(hidden_size, hidden_size)
        self.matchatt = MatchingAttention(num_features+hidden_size, num_features+hidden_size, att_type='general2')
        self.linear = nn.Linear(num_features+hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.smax_fc = nn.Linear(hidden_size, num_classes)
        self.no_cuda = no_cuda
        
        if self.use_contrastive:
            self.projection_head = nn.Sequential(
                nn.Linear(num_features+hidden_size, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, hidden_size//2)
            )
            self.contrastive_loss = ContrastiveLoss()

    def forward(self, x, edge_index, edge_norm, edge_type, seq_lengths, umask, nodal_attn, avec, labels=None):
        out = self.conv1(x, edge_index, edge_type)
        out = self.conv2(out, edge_index)
        emotions = torch.cat([x, out], dim=-1)
        
        # Projection for contrastive learning
        if self.use_contrastive and labels is not None:
            projections = self.projection_head(emotions)
            contrast_loss = self.contrastive_loss(projections, labels)
        else:
            contrast_loss = torch.tensor(0.0).to(x.device)
        
        log_prob = classify_node_features(emotions, seq_lengths, umask, 
                                          self.matchatt, self.linear, 
                                          self.dropout, self.smax_fc, 
                                          nodal_attn, avec, self.no_cuda)
        
        return log_prob, contrast_loss
```

## 3. Modification du DialogueGCNModel (lignes 878-1030)

```python
class DialogueGCNModel(nn.Module):
    def __init__(self, ..., use_contrastive=False):
        ...
        self.use_contrastive = use_contrastive
        self.graph_net = GraphNetwork(2*D_e, n_classes, n_relations, max_seq_len, 
                                      graph_hidden_size, dropout, self.no_cuda,
                                      use_contrastive=self.use_contrastive)
        ...

    def forward(self, U, qmask, umask, seq_lengths, labels=None):
        ...
        log_prob, contrast_loss = self.graph_net(features, edge_index, edge_norm, 
                                                 edge_type, seq_lengths, umask, 
                                                 self.nodal_attention, self.avec,
                                                 labels=labels)
        
        return log_prob, edge_index, edge_norm, edge_type, edge_index_lengths, contrast_loss
```

## 4. Modification de la fonction d'entraînement

```python
def train(model, data_loader, optimizer, device, contrastive_weight=0.5):
    model.train()
    total_loss = 0
    
    for batch in data_loader:
        U, qmask, umask, seq_lengths, labels = batch
        U, qmask, umask = U.to(device), qmask.to(device), umask.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        log_prob, _, _, _, _, contrast_loss = model(U, qmask, umask, seq_lengths, labels)
        
        # Calculate losses
        nll_loss = F.nll_loss(log_prob, labels)
        
        if model.use_contrastive:
            loss = nll_loss + contrastive_weight * contrast_loss
        else:
            loss = nll_loss
            
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(data_loader)
```

## Points Clés d'Intégration

### Projection Head (lignes 740-745):
- Ajout d'un petit réseau pour projeter les features dans un espace où l'apprentissage contrastif sera appliqué

### Contrastive Loss (lignes 690-720):
- Nouvelle loss qui maximise la similarité entre échantillons de même classe
- Minimise la similarité entre échantillons de classes différentes

### Modifications du Forward Pass (lignes 750-760 et 1010-1020):
- Retourne à la fois la loss classique et la loss contrastive
- Permet de combiner les deux losses pendant l'entraînement

### Hyperparamètre (ligne 1050):
- `contrastive_weight` contrôle l'importance relative de la loss contrastive

## Avantages pour les Émotions Proches

| Critère | Amélioration |
|----------|--------------|
| Meilleure Séparation | Les embeddings des émotions similaires (frustration/colère) seront plus distincts |
| Robustesse | Le modèle apprend des caractéristiques plus discriminatives |
| Flexibilité | Le poids contrastif peut être ajusté selon le jeu de données |

Cette intégration permet au modèle de mieux distinguer les émotions proches tout en conservant sa structure originale.